In [30]:
from scipy.integrate import cumtrapz
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

%matplotlib inline
plt.style.use('seaborn-poster')

# Constants
mu = 1.2566 * 10 ** (-6)        # Vacuum permeability in H/m
c = 3 * 10 ** 8                 # Speed of light in m/s

# Code parameters
R_out = 0.1                     # Radius of larger coil in m
R_in = 0.075                    # Radius of smaller coil in m
I_out = 10                      # Amplitude of current
I_in = 15                       # Amplitude of current
f = 100000                      # Frequency of current in kHz

# Time, current and dI/dt
T = np.arange(0,2,1/f)      # Time range and step size
I = np.cos(2 * np.pi * f * T)
DI = -2 * np.pi * f * np.sin(2 * np.pi * f * T)


def interact_fxn(z, t):
    def Bfield(x,y,R):
        """_summary_

        Args:
            x (float): X position of the point experiencing the field
            y (float): Y position of the point experiencing the field
            R (float): radius of the coil producing the field

        Returns:
            float: Magnitude of B field in Z direction experienced at (x,y,z)
        """
        Bz = np.array([])                # Magnetic field

        # Angle phi (polar angle from 0 to 2pi)
        Phi = np.arange(0, 2*np.pi, 0.01) 
        DBz = np.array([])

        for phi in Phi:
            # retarded time t' = t-R/c
            tr = int(f * (t - ((R * np.cos(phi)-x)**2 + (R * np.sin(phi) - y)**2 + z**2) / c))

            # static component of B that depends on I
            dBz_I = I[tr] * R * (R - x * np.cos(phi) - y * np.sin(phi))/((R * np.cos(phi)-x)**2 + (R * np.sin(phi) - y)**2 + z**2)**(3/2)

            # x and y direction static fields 
            # dBx = (R * np.cos(phi) * z)/((R * np.cos(phi)-x)**2 + (R * np.sin(phi) - y)**2 + z**2)**(3/2)
            # dBy = (R * np.sin(phi) * z)/((R * np.cos(phi)-x)**2 + (R * np.sin(phi) - y)**2 + z**2)**(3/2)

            # time dep. term of B that depends on dI/dt
            dBz_dI = DI[tr] * R / c * (R - x * np.cos(phi) - y * np.sin(phi))/((R * np.cos(phi)-x)**2 + (R * np.sin(phi) - y)**2 + z**2)
            DBz = np.append(DBz, dBz_I + dBz_dI)

        # integrate function over the range of phi 
        Bz = np.append(Bz, mu / 4 / np.pi * cumtrapz(DBz, Phi)[-1])
        return Bz


    # 2D plot along x = 0
    X = np.arange(-0.3,0.32,0.02) 
    y = 0
    Bz = np.array([I_out * Bfield(x,y,R_out) - I_in * Bfield(x,y,R_in) for x in X])
    plt.plot(X, Bz)
    plt.xlabel('Position X (m)')
    plt.ylabel('B Field in Z Direction')
    plt.title('Bz at z=%s, for Coil R_in = %s, R_out = %s' % (z, R_in, R_out))
    plt.show()

    # 3D surface plot
    X = np.arange(-0.2,0.22,0.02)    # Range of field we are simulating over
    Y = np.arange(-0.2,0.22,0.02) 
    X, Y = np.meshgrid(X, Y)
    Bz = np.array([I_out * Bfield(x,y,R_out) - I_in * Bfield(x,y,R_in) for x,y in zip(np.ravel(X), np.ravel(Y))]).reshape(X.shape)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title('Bz at z=%s, for Coil R_ini = %s, R_out = %s' % (z, R_in, R_out))
    ax.plot_surface(X, Y, Bz)
    ax.set_xlabel('Position X (m)', labelpad=20)
    ax.set_ylabel('Position Y (m)', labelpad=20)
    plt.show()

interact(interact_fxn, z=(0,0.5,0.1), t = (0.1, 0.2, 0.02))

interactive(children=(FloatSlider(value=0.2, description='z', max=0.5), FloatSlider(value=0.14, description='t…

<function __main__.interact_fxn(z, t)>